# Practical Lab 2: Multivariate Linear Regression, Non-Parametric Models and Cross-Validation  
**Student Name:** Yogesh Kumar Gopal  
**ID:** 8996403
**Course:** CSCN8010


### Part 1: Data Preparation & EDA

In [ ]:
# Import necessary libraries
from sklearn import datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Get the Data

In [ ]:
# Load diabetes dataset as DataFrame
diabetes = datasets.load_diabetes(as_frame=True)
df = diabetes.frame
df.head()


**Objective:** Predict "disease progression one year after baseline" (target variable `y`) based on patient features (X).

EDA - Exploratory Data Analysis

In [ ]:
# Statistics
df.describe()

In [ ]:
# Scatter plot for BMI vs Disease Progression
plt.scatter(df['bmi'], df['target'])
plt.xlabel('BMI')
plt.ylabel('Disease Progression')
plt.title('BMI vs Disease Progression')

# Correlation heatmap
plt.figure(figsize=(10,8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')


## Insights based on your EDA plots:

1. BMI is strongly correlated with disease progression:

    The scatter plot of BMI vs. target (disease progression) shows a clear positive trend, indicating that higher BMI values are associated with increased disease progression.

    The correlation matrix confirms this with a strong correlation coefficient of 0.59 between BMI and the target variable — one of the highest correlations observed.

2. Other significant features include S5 and BP:

    S5 shows a strong positive correlation (0.57) with the target.

    Blood Pressure (bp) also shows a moderate positive correlation (0.44), suggesting it’s a relevant predictor.

3. Feature distributions are standardized and mostly symmetric:

    Most features like age, bmi, bp, and s1–s6 are centered around 0 due to standardization, with fairly normal distributions.

    The target variable (disease progression) is right-skewed, indicating more cases with lower progression values and fewer extreme progression cases.

4. Some features are highly correlated with each other:

    S1 and S2 (both likely related to cholesterol or serum levels) have a very strong positive correlation (0.9), indicating multicollinearity.

    S3 and S4 are negatively correlated (-0.74), which may impact model interpretation.